# CSC421 Assignment 4 - Part II Hidden Markov Models (5 points) #
### Author: George Tzanetakis 

This notebook is based on the supporting material for topics covered in **Chapter 15 Probabilistic Reasoning over Time** from the book *Artificial Intelligence: A Modern Approach.* 

This part relies on the provided notebook code probability.ipynb, viterbi.ipynb as well as the notebook for Markov Chains and HMMs that is under Resources and was presented in class (this notebook requires the hmmlearn Python package). 

```
Misunderstanding of probability may be the greatest of all impediments
to scientific literacy.

Gould, Stephen Jay
```



## Introduction 

Hidden Markov Models are probabilistic models that capture uncertainty over time. In this assignment you will become familiar with the underlying concepts by using two different implementations of HMMs - one based on hmmlearn and the other provided by the book authors. 

## Question 4.2A (Minimum) CSC421 -  (1 point, CSC581C - 0 points) 

Use the umbrella HMM described by the book and implemented in the probability.ipynb notebook with the code in the Markov Chain and HMM notebook. Use the hmmlearn version rather than the one that is implemented from "scratch". Generate 30 random samples and display with characters both the hidden states and observations. More specifically change the state2color and obs2color functions to display characters instead of colors. Use R and S for a rainy day and a not rainy (sunny) day and u and underscore(_) for the umbrella. So for example: 

Ru,Ru,R_,S_,Su 
Would correspond to: 
```
Day1: Rainy with umbrella 
Day2: Rainy with umbrella 
Day3: Rainy with no umbrella 
Day4: Sunny with no umbrella 
Day5: Sunny with umbrella 
```

In [8]:
import numpy as np
from hmmlearn import hmm
from probability import *
from utils import rounder

In [7]:
model = hmm.MultinomialHMM(n_components=2)
model.startprob_ = np.array([1.0, 0.0])
model.transmat_ = np.array([[0.7, 0.3], [0.2, 0.8]])
model.emissionprob_ = np.array([[0.6, 0.3, 0.1, 0.0], [0.0, 0.1, 0.4, 0.5]])

X, Z = model.sample(30)
samples = [i for x in X for i in x]

obj2color = {} 
obj2color[0] = 'S_'
obj2color[1] = 'Su'
obj2color[2] = 'R_'
obj2color[3] = 'Ru'

print([obj2color[s] for s in samples])

['Su', 'Su', 'S_', 'S_', 'S_', 'Ru', 'S_', 'S_', 'Su', 'S_', 'Su', 'S_', 'S_', 'S_', 'S_', 'S_', 'R_', 'Su', 'R_', 'Ru', 'Su', 'Ru', 'Ru', 'Ru', 'Ru', 'R_', 'Ru', 'Ru', 'Ru', 'S_']


## Question 4.2B (Minimum) (CSC421 - 1 point, CSC581C - 0 point) 


Express the weather example in terms of a transition and sensor model following the conventions of the HMM implementation provided in probability.ipynb from the AIMA authors. The implementation of HMMs in the AIMA notebook is limited and only supports a sensor model with a binary variable. Convert the 4 sky colors to a binary variable as follows (you can think of it as blue sky (True or False): 

```
[0.6, 0.3, 0.1, 0.0] will become ]0.9, 0.1] 
amd 
[0.0, 0.1, 0.4, 0.5] will become [0.1, 0.9] 
```

Suppose you have the following evidence for Blue Sky: 
```
evidence = [True, False, False, True, True]
```

What is the maximum likelihood sequence of hidden states (sunny or cloudy) for this evidence? 
Calculate and display the answer using the viterbi algorithm from viterbi.ipynb. 


In [11]:
transition_model = [[0.7, 0.3], [0.2, 0.8]] 
sensor_model = [[0.9, 0.1], [0.1, 0.9]]
evidence = [True, False, False, True, True]

model = HiddenMarkovModel(transition_model, sensor_model)
print(rounder(viterbi(model, evidence)))

([1, 0, 0, 1, 1], [0.8804, 0.1585, 0.1141, 0.0308, 0.0194])


# Question 2C (Expected) 1 point 

Consider the umbrella HMM implemented using hmmlearn as you did in question 4.2A. Generate 1000 samples i.e you will have a sequence of 1000 states and corresponding sequence of 10000 observations. One way to view this data is to consider it as a classification problem where your feature is whether there is an umbrella or not and your predicted class is whether it is rainy or not. 

Write a function called classification_accuracy that takes as input two lists of states and returns the number of states that are the same in both lists as a percentage. 

Now consider the sequence of states you generated as ground truth. Using the specified umbrella HMM predict what sequence of states is the most likelily to have generated the generated sequence of observations. This is similar to the visual comparison of the original and predicted states in the provided notebook. Here we quantify this. 

What is the classification accuracy in this case? 

Now consider a new HMM in which the transition model for the umbrella example has 0.5 for all probability values. 
Effectively this disregards any temporal dependenices and each time step is decided independently. In fact it corresponds to a Naive Bayes classifier with a single feature which is whether umbrella is true or false. 

What is the classification accuracy in this case? 


In [23]:
def classification_accuracy(X, Z):
    X = [i for x in X for i in x]
    error = 0
    for x, z in zip(X, Z):
        if x % 2 == 0 and z == 1:
            error += 1
        elif x % 2 != 0 and z == 0:
            error += 1
    return (len(Z) - error) / len(Z)

transition_models = [np.array([[0.7, 0.3], [0.2, 0.8]]), np.array([[0.5, 0.5], [0.5, 0.5]])]
for transition_model in transition_models:
    model = hmm.MultinomialHMM(n_components=2)
    model.startprob_ = np.array([1.0, 0.0])
    model.transmat_ = transition_model
    model.emissionprob_ = np.array([[0.6, 0.3, 0.1, 0.0], [0.0, 0.1, 0.4, 0.5]])
    X, Z = model.sample(1000)
    print(f'classification accuracy: {classification_accuracy(X, Z)}')


classification accuracy: 0.673
classification accuracy: 0.636


# QUESTION 2D (Expected ) 1 point

Modify the code provided in the Markov Chain and HMM notebook so that innstead of 4 sky colors as the sensor model a continuous single Gauassian distribution is used. For sunny days the Gaussian distribution should have a mean of 20 degrees with a standard deviation of 5 and for cloudy days the Gauassian distribution should have a mean of 14 degrees with a standard deviation of 3. Show 20 generates samples using characters. 


C12,C15,S21,S22 
Would correspond to: 
```
Day1: Cloudy and 12 
Day2: Cloudy and 15 
Day3: Sunny and 21   
Day4: Sunny and 22
```

Do a month and year visualization of the temperature using the Plasma colormap (make sure you normalize the temperature values first): 

https://matplotlib.org/3.2.0/tutorials/colors/colormaps.html

In [12]:
0 % 2

0

# QUESTION 2E (Advanced) 1 point 


Show how you can learn the HMM for the continuous random variable case. Use 10000 samples 
generated using the code you wrote for the previous question. 

The learn the parameters of an HMM using a Gaussian HMM: 
https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.GaussianHMM


Compare the original transition and sensor model with the estimated transition and sensor model similarly to what was done for the multinomial HMM with the discrete sensor model. 

In [8]:
# YOUR CODE GOES HERE 


# QUESTION 2F (ADVANCED) (CSC421 - 0 points, CSC581C - 2 points)

This question is a bit more open ended, will require some creativity and extra work. Consider the following problem: during your day your cell phone collects location data in terms of x,y coordinates. You do different activities such as going to university, eating, going to the gym. These activities take place in particular locations such as Restaurant A and Restaurant B or Gym A, Gym B and each particular location can be thought of as a two-dimensional Gaussian distribution of location points. If you consider the activity as the hidden state and the location as the observation you have a Hidden Markov Model. Because activities take place in multiple locations you can model this as a Gaussian Mixture Model (GMM). Each Gaussian will be multivariate 2D Gaussian distribution characterized by two means and and a 2 by 2 covariance matrix. 

Consider a hypothetical scenario with 3 activities (eat, study, exercise) and 3 locations (GMM components) for each activity. You will need to do some reading about how GMMs work. 

Basically the goal is the follow the format of the Markov Chain and HMM notebook and create appropriate visualizations using this problem. 

1. Visualize on a 2D plane using circles the different locations and corresponding mixture components 
2. Generate a dataset using a Hidden Markov Model of the problem 
3. Visualize the dataset on a 2D plane 
4. Show how you can learn the parameters of this HMM using https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.GMMHMM

In [9]:
# YOUR CODE GOES HERE 